In [ ]:
import numpy as np
from model_functions import *
from policy_utils import *
import matplotlib.pyplot as plt
import gymnasium as gym

from algorithms import *
from FrozenLake import *
import pandas as pd
import constants
import math
from policy_utils import *
import torch
import mlflow
from CurriculumPMPO import CurriculumPMPO
import optuna
from experiment_result_utils import *
from env_register import *
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO
from TMDP_v1 import *
nrows = 6
nS = nrows**2
nA = 4
seed = constants.SEEDS[0]
set_policy_seed(seed)
num_bins = 10
gamma = .9999999
tau = 0.
dense_reward = True
reward_shape = True
shape_range=(-1,0)
goal_reward = 1.
is_slippery=False
env = FrozenLakeEnv(is_slippery=is_slippery, seed=seed, 
                        desc=generate_random_map(nrows, seed=seed), 
                        reward_shape=reward_shape,
                        num_bins=num_bins,
                        dense_reward=dense_reward,
                        shape_range=shape_range,
                        goal_reward=goal_reward,
                        )
xi = np.ones(env.nS) * 1/env.nS


In [ ]:
env_kwargs = dict(env=env, xi=xi, tau=tau, gamma=gamma)
tmdp = TMDP(**env_kwargs)

In [ ]:
vec_env = DummyVecEnv([lambda: tmdp])
model = PPO("MlpPolicy", vec_env, verbose=1, learning_rate=0.2)
model.learn(total_timesteps=40000, callback=TeleportFilterCallback())

In [ ]:
obs = vec_env.reset()
done = False
truncated = False
tmdp.env.render_mode = 'human'
step = 0
while not done and not truncated:
    action, _states = model.predict(obs, deterministic=True)
    print(action)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render()
    step += 1
    if step > 100:
        truncated = True
#vec_env.close()

In [ ]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

class PPOWithBatchAndTimestepLimit(PPO):
    def __init__(self, *args, max_batches=None, **kwargs):
        super(PPOWithBatchAndTimestepLimit, self).__init__(*args, **kwargs)
        self.max_batches = max_batches
        self.total_timesteps_tracked = 0

    def learn(self, total_timesteps, callback=None, log_interval=1, tb_log_name="PPO", reset_num_timesteps=True, progress_bar=False):
        assert self.max_batches is not None, "You must set max_batches to use this custom training loop."
        batch_count = 0

        while self.total_timesteps_tracked < total_timesteps:
            # Collect rollouts using the current policy
            rollout = self.collect_rollouts(self.env, callback, self.rollout_buffer, n_rollout_steps=self.n_steps)
            
            if rollout.continue_training is False:
                break

            self._update_current_progress_remaining(self.total_timesteps_tracked, total_timesteps)

            # Perform PPO update
            self.train()

            batch_count += 1
            self.total_timesteps_tracked += self.n_steps * self.env.num_envs

            # Log training information
            if batch_count % log_interval == 0:
                self._dump_logs(batch_count)
            
            if batch_count >= self.max_batches:
                print(f"Batch limit reached: {batch_count} batches processed.")
                break

        return self

# Example usage
if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    env = DummyVecEnv([lambda: env])

    total_timesteps = 10000
    max_batches = 100

    model = PPOWithBatchAndTimestepLimit("MlpPolicy", env, max_batches=max_batches, verbose=1)

    # Train the model for a specific number of timesteps, calling learn repeatedly if necessary
    while model.total_timesteps_tracked < total_timesteps:
        remaining_timesteps = total_timesteps - model.total_timesteps_tracked
        model.learn(total_timesteps=remaining_timesteps)

    print(f"Total timesteps after training: {model.total_timesteps_tracked}")
